In [3]:
import numpy as np
import cv2
import os
import glob
import warnings
import matplotlib.pyplot as plt

In [4]:
def bb_midpoint_to_corner(bb):
    label = bb[0]
    x1 = bb[1] - bb[3]/2
    x2 = bb[1] + bb[3]/2
    y1 = bb[2] - bb[4]/2
    y2 = bb[2] + bb[4]/2
    # A: area will only be used for sorting
    area = bb[3]*bb[4]
    corner_list = [label, x1, x2, y1, y2, area]
    return np.array(corner_list)

def open_yolo_sort(path, image_name):
    try:
        image = cv2.imread(path + image_name)
        shape = image.shape
        width = shape[1]
        height = shape[0]
        label = path + os.path.splitext(image_name)[0] + ".txt"
        boxes = np.genfromtxt(label, delimiter=' ')
        bb = boxes
        # reshaping the np array is necessary in case a file with a single box is read
        boxes = boxes.reshape(boxes.size//5, 5)
        #print(boxes.shape)
        boxes = np.apply_along_axis(bb_midpoint_to_corner, axis=1, arr=boxes)
        # A: sorting by area
        boxes = boxes[boxes[:, 5].argsort()]
        # A: reversing the sorted list so bigger areas come first
        boxes = boxes[::-1]
        return image, boxes, width, height
    except Exception as e:
        #print(e)
        print(image_name)
        return image, None, None, None
    

colors = [(162, 0, 255),  # Chave seccionadora lamina (Aberta)
         (97, 16, 162),   # Chave seccionadora lamina (Fechada)
         (81, 162, 0),    # Chave seccionadora tandem (Aberta)
         (48, 97, 165),   # Chave seccionadora tandem (Fechada)
         (121, 121, 121), # Disjuntor
         (255, 97, 178),  # Fusivel
         (154, 32, 121),  # Isolador disco de vidro
         (255, 255, 125), # Isolador pino de porcelana
         (162, 243, 162), # Mufla
         (143, 211, 255), # Para-raio
         (40, 0, 186),    # Religador
         (255, 182, 0),   # Transformador
         (138, 138, 0),   # Transformador de Corrente (TC)
         (162, 48, 0),    # Transformador de Potencial (TP)
         (162, 0, 96)     # Chave tripolar
         ] 

colors_rgb = []
for c in colors:
    colors_rgb.append((c[2], c[1], c[0]))

In [7]:
_ITER_COUNT = 10

def grabcut(image_path, save_path, image_name, filter_list):
    """Use Grabcut to create a binary segmentation of an image within a bounding box
    Param:
        image_path: path to the dir where the images are stored
        save_path: path to the dir where the masks will be saved
        image_name: name for the image proper
        filter_list: array with the labels that will be accounted for. This version
        to only really work with one, though
    Returns:
        1 if there were any labels of the class from filter list and the mask was made
        succesfully, in which case it will be saved using save_path/image_name.png.
    Based on:
        https://stackoverflow.com/questions/12810405/opencv-set-color-to-a-foreground-marked-pixel-gc-pr-fgd
        https://docs.opencv.org/3.4/d8/d83/tutorial_py_grabcut.html
        https://pyimagesearch.com/2020/09/28/image-segmentation-with-mask-r-cnn-grabcut-and-opencv/     
    """
    image, bb, w, h = open_yolo_sort(image_path, image_name)
    mask = image.copy()*0
    # the mask has to be in grayscale for grabcut to work
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
#     fig = plt.figure(figsize=(20,20))
#     ax = fig.add_subplot(141)
#     ax.imshow(image)
#     plt.axis('off')
    filter_flag = False
    if bb is not None:
        for label, x1, x2, y1, y2, area in bb:
            if label in filter_list:
                filter_flag = True
                rw = (x2*w - x1*w)
                rh = (y2*h - y1*h)
                # the likely foreground around the center
                cv2.rectangle(mask, (int(x1*w), int(y1*h)), (int(x2*w), int(y2*h)), 100, -1)
                # the certainly foreground will be the inntermost 20%
                cv2.rectangle(mask, (int(x1*w + rw*0.4), int(y1*h + rh*0.4)), (int(x2*w - rw*0.4), int(y2*h - rh*0.4)), 255, -1)
    else:
        cv2.imwrite(save_path + os.path.splitext(image_name)[0] +".png", image*0)
        return 1
        
    # grabcut will only be performed if there was any bounding box in the filter list to start with
    if filter_flag:
        # https://stackoverflow.com/questions/12810405/opencv-set-color-to-a-foreground-marked-pixel-gc-pr-fgd
        # https://docs.opencv.org/3.4/d8/d83/tutorial_py_grabcut.html
        # https://pyimagesearch.com/2020/09/28/image-segmentation-with-mask-r-cnn-grabcut-and-opencv/
        # GC_BGD Certainly a background pixel
        # GC_FGD Certainly a foreground (object) pixel
        # GC_PR_BGD Probably a background pixel
        # GC_PR_FGD Probably a foreground pixel
#         ax2 = fig.add_subplot(142)
#         ax2.imshow(cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB))
#         plt.axis('off')
        mask[mask == 255] = cv2.GC_FGD    # the 20% innermost
        mask[mask == 100] = cv2.GC_PR_FGD # the area around center
        mask[mask == 0]   = cv2.GC_BGD    # the black pixels background
        gct = np.zeros(image.shape[:2], np.uint8)
        bgdModel, fgdModel = np.zeros((1, 65), np.float64), np.zeros((1, 65), np.float64)
        (gcMask, bgModel, fgModel) = cv2.grabCut(image, mask, None, bgdModel, fgdModel, _ITER_COUNT, cv2.GC_INIT_WITH_MASK)
        outputMask = np.where((gcMask == cv2.GC_BGD) | (gcMask == cv2.GC_PR_BGD), 0, 1)
        outputMask = (outputMask * 255).astype("uint8")
#         ax3 = fig.add_subplot(143)
#         ax3.imshow(cv2.cvtColor(outputMask, cv2.COLOR_GRAY2RGB))
#         plt.axis('off')
        output = cv2.cvtColor(outputMask, cv2.COLOR_GRAY2BGR)
        output = np.where(output != (0, 0, 0), colors_rgb[filter_list[0]], output)
        cv2.imwrite(save_path + os.path.splitext(image_name)[0] +".png", output)
        return 0
    else:
        print(os.path.splitext(image_name)[0])
        cv2.imwrite(save_path + os.path.splitext(image_name)[0] +".png", image*0)
        return 1
#         output = cv2.bitwise_and(image, image, mask=outputMask)
        #mask = (mask * 255).astype("uint8")
    #     image_copy = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
    #     with open('image.csv', 'w') as f:
    #         f.write(np.array2string(mask, precision=2, separator=',', suppress_small=True))
        #cv2.imwrite("image_test.png", mask)
#         ax3 = fig.add_subplot(144)
#         ax3.imshow(output)
#         plt.axis('off')
#         plt.show()

In [ ]:
paths = ["../data/train/"]

# filter list está no modo in
filter_list = [10] # religador
save_path = "../data/recloser/train_gci/"

import sys
np.set_printoptions(threshold=sys.maxsize)

check_path = os.path.isdir(save_path)
if not check_path:
    os.makedirs(save_path)
    print("created folder: ", save_path) 
    
#label_path = "../data/labels_17class/"
count = 0
for image_path in paths:
    file_list = os.path.join(os.path.join(image_path, "*.*"))
    image_list = []
    file_list = glob.glob(file_list)
    for name in file_list:
        if "txt" not in name:
            image_list.append(name.split("/")[-1])

    for file in image_list:
        count+= grabcut(image_path, save_path, file, filter_list)

2022-07-01_p4_P4_C053_X1433_Y0_Z-10_A-1251_E180_rgb
P3_C42_X1273_Y0_Z-12_A659_E620_rgb_HD_resize_rotate_lFx8HsY
2022-07-05_p4_P4_C022_X804_Y0_Z-36_A1306_E410_rgb_HD
FLIR5790_rgb_YoQndvz
FLIR8645_rgb_0Ny0JBV
FLIR8258_rgb_3es3t5k
FLIR7497_rgb_OmM399I
2022-07-04_p1_P1_C044_X1434_Y0_Z-3_A600_E700_rgb_HD
2022-07-05_p4_P4_C056_X1435_Y0_Z19_A-1748_E350_rgb
20210811_094519
FLIR8178_rgb_OeAC2yE
2022-07-14_p4_P4_C042_X1274_Y0_Z10_A661_E620_rgb_HD
FLIR7496_rgb_ZSGcwKb
FLIR7636_rgb
FLIR1605_rgb
FLIR7357_rgb
2022-07-01_p1_P1_C044_X1433_Y0_Z34_A603_E700_rgb
FLIR6448_rgb
FLIR6525_rgb
20210805_095923
2022-07-05_p4_P4_C038_X1123_Y0_Z-10_A1309_E450_rgb_HD
20210811_094731
FLIR7241_rgb
FLIR7204_rgb
FLIR6234_rgb
FLIR6534_rgb
P3_C43_X1433_Y0_Z32_A403_E700_rgb_HD_resize_rotate
FLIR6445_rgb
FLIR6938_rgb
2022-07-05_p4_P4_C041_X1268_Y0_Z10_A1681_E110_rgb
FLIR6230_rgb
FLIR7271_rgb
FLIR6332_rgb
FLIR7256_rgb
img_20220405_165646_776_rgb
2022-07-13_p3_P3_C042_X1274_Y0_Z40_A664_E620_rgb
2022-07-05_p3_P3_C025_X1125_Y0

In [ ]:
print(f'Total images: {len(image_list)}')
print(f'Images with no recloser: {count}')